In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from tqdm import tqdm
import random, sys, os

In [3]:
import sparse_linear_lib as sll

In [4]:
device = torch.device("cuda:1")

In [5]:
import pandas as pd
import time

## Pair Linear approximation

In [6]:
# Ns = [16, 64, 256, 1024, 4096] #, 16384]
Ns = [256, 1024, 4096] #, 16384]
seeds = [147, 258, 369, 321, 654, 987, 741, 852, 963, 159, 357, 951, 753]

In [7]:
# Ns = [16384]
# seeds = [147]

In [8]:
mse = nn.MSELoss()

In [9]:
TRAIN_STEPS = 20000 #2000
def train_model(model, optimizer, X, A):
    for i in range(TRAIN_STEPS):
        out = model(X)
        loss = mse(out, A)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i%1000 == 0:
            print(f"The MSE loss is : {float(mse(out,A))}")
            
    with torch.no_grad():
        start = time.time()
        out = model(X)
        tt = time.time()-start
    return out, tt

In [10]:
def get_svd_output(A, n_comp):
    U, S, V = torch.svd(A)

    with torch.no_grad():
        _U = U[:, :n_comp]
        _S = S[:n_comp]
        _V = V[:, :n_comp]
        
        start = time.time()
        out = torch.mm(torch.mm(_U, torch.diag(_S)), _V.t())
        tt = time.time()-start
    
#     S[n_comp:] *= 0
#     out = torch.mm(torch.mm(U, torch.diag(S)), V.t())
    return out, tt

In [11]:
def save_stats(df, out, A, method, seed, nparam, tim, filename):
    diff = (out.data-A).abs()
    
    mean, std = float(diff.mean()), float(diff.std())
    err = float(mse(out, A))

    df = df.append({"method":method, "seed":seed, "mse":err,
                    "mean":mean, "std":std, "params":nparam, "time":tim}, 
                   ignore_index=True)
    df_ = df.copy()
    df.to_csv(f"./outputs/{file_name}.csv")
    
    print(f"Saving... file:{file_name} method:{method}")
    return df_

In [12]:
# !mkdir outputs/

In [13]:
df = pd.DataFrame(columns=['mse', 'mean', 'std'])
df

,mse,mean,std


In [14]:
class Add_PairLinears(nn.Module):
    
    def __init__(self, input_dim, num_adds):
        super().__init__()
        self.pair_mixers = []
        self.perm_indices = []
        for i in range(num_adds):
            m = sll.PairLinear_MixerBlock(input_dim, input_dim)
            self.pair_mixers.append(m)
            if i > 0:
                rm = torch.randperm(input_dim)
                self.perm_indices.append(rm)
                
        self.pair_mixers = nn.ModuleList(self.pair_mixers)
        
    def forward(self, x):
        y = torch.zeros_like(x)
        for i, m in enumerate(self.pair_mixers):
            if i > 0:
                _x = x[:, self.perm_indices[i-1]]
            else:
                _x = x
                
            y += m(_x)
        return y

In [15]:
# model = Add_PairLinears(N, 4).to(device)

In [16]:
class Stack_PairLinears(nn.Module):
    
    def __init__(self, input_dim, num_adds):
        super().__init__()
        self.pair_mixers = []
        self.perm_indices = []
        for i in range(num_adds):
            m = sll.PairLinear_MixerBlock(input_dim, input_dim)
            self.pair_mixers.append(m)
            if i > 0:
                rm = torch.randperm(input_dim)
                self.perm_indices.append(rm)
                
        self.pair_mixers = nn.ModuleList(self.pair_mixers)
        
    def forward(self, x):
        for i, m in enumerate(self.pair_mixers):
            if i == 0:
                x = m(x)
            else:
                x = m(x[:, self.perm_indices[i-1]])
        return x

In [17]:
def log_base(a, base):
    return np.log(a) / np.log(base)

In [18]:
# df.append({"mse":1.0, "mean":2.0, "std":4.0}, ignore_index=True)

In [19]:
# import warnings
# warnings.filterwarnings(action='once') 

import warnings
warnings.filterwarnings('ignore')

'''
FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"method":method, "seed":seed, "mse":err,
'''

'\nFutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.\n  df = df.append({"method":method, "seed":seed, "mse":err,\n'

In [20]:
A = None
for N in Ns:
# for N in [4096]:
#     torch.cuda.empty_cache()
    X = torch.eye(N).to(device)
    
    df = pd.DataFrame(columns=['method', 'seed', 'mse', 'mean', 'std', 'params', 'time'])
    file_name = f'record_err_{N}'
        
#     for SEED in [147]:
    for SEED in seeds:
#         torch.cuda.empty_cache()
        print()
        print(f"Experiment N={N} SEED={SEED}")
        
        torch.manual_seed(SEED)
        
        del A
        
        A = torch.rand(N, N).to(device)*2-1
        ### For each method compute the stats
        
        #####################################################
        ##### First 2x2 factorization
        model = sll.PairLinear_MixerBlock(N, N).to(device)        
        optimizer = torch.optim.Adam(model.parameters(), lr=0.003)
        
        #### Train model
        out, tim = train_model(model, optimizer, X, A)
        _n_params = sum(p.numel() for p in model.parameters())
        df = save_stats(df, out, A, 'pair', SEED, _n_params, tim, file_name)
        #####################################################
        del model, optimizer
        
        #####################################################
        ##### First log(N) factorization
#         _m = int(np.ceil(np.log2(N)))

        #### sqrt(N)/2 
        _m = int(np.ceil(np.sqrt(N)/2))
        model = sll.BlockLinear_MixerBlock(N, _m).to(device)        
        optimizer = torch.optim.Adam(model.parameters(), lr=0.003)
        
        #### Train model
        out, tim = train_model(model, optimizer, X, A)
        _n_params = sum(p.numel() for p in model.parameters())
        df = save_stats(df, out, A, 'block-sqrt-half', SEED, _n_params, tim, file_name)
        #####################################################
        del model, optimizer
        
        
        #####################################################
        ##### Second sqrt(N) factorization
        _m = int(np.ceil(np.sqrt(N)))
        model = sll.BlockLinear_MixerBlock(N, _m).to(device)        
        optimizer = torch.optim.Adam(model.parameters(), lr=0.003)
        
        #### Train model
        out, tim = train_model(model, optimizer, X, A)
        _n_params = sum(p.numel() for p in model.parameters())
        df = save_stats(df, out, A, 'block-sqrt', SEED, _n_params, tim, file_name)
        #####################################################
        del model, optimizer
        
        
        #####################################################
        _m = int(np.ceil(_n_params/(N*2)))
        out, tim = get_svd_output(A, _m)

        n_params = N*_m*2
        df = save_stats(df, out, A, 'lowR-same-param', SEED, n_params, tim, file_name)
        #####################################################
        
        #####################################################
        _m = int(np.ceil(_n_params/N))
        out, tim = get_svd_output(A, _m)

        n_params = N*_m
        df = save_stats(df, out, A, 'lowR-samex2', SEED, n_params, tim, file_name)
        #####################################################
        
        #####################################################
        _m = N // 2
        out, tim = get_svd_output(A, _m)

        n_params = N*_m*2
        df = save_stats(df, out, A, 'lowR-half', SEED, n_params, tim, file_name)
        #####################################################
        
        #####################################################
#         torch.cuda.empty_cache()
        if N > 1024: continue

        ##### Pair Linear models parallel addition
        _m = int(np.ceil(np.log2(N)))
        model = Add_PairLinears(N, _m).to(device)        
        optimizer = torch.optim.Adam(model.parameters(), lr=0.003)
        
        #### Train model
        out, tim = train_model(model, optimizer, X, A)
        _n_params = sum(p.numel() for p in model.parameters())
        df = save_stats(df, out, A, 'pair-Add', SEED, _n_params, tim, file_name)
        #####################################################
        del model, optimizer
        
        #####################################################
        
        ##### Pair Linear models sequential composition
        _m = int(np.ceil(np.log2(N)))
        model = Stack_PairLinears(N, _m).to(device)        
        optimizer = torch.optim.Adam(model.parameters(), lr=0.003)
        
        #### Train model
        out, tim = train_model(model, optimizer, X, A)
        _n_params = sum(p.numel() for p in model.parameters())
        df = save_stats(df, out, A, 'pair-Seq', SEED, _n_params, tim, file_name)
        #####################################################
        del model, optimizer
#         torch.cuda.empty_cache()


Experiment N=256 SEED=147
The MSE loss is : 0.33718395233154297
The MSE loss is : 0.30751633644104004
The MSE loss is : 0.3056437373161316
The MSE loss is : 0.30491209030151367
The MSE loss is : 0.3048853278160095
The MSE loss is : 0.30488526821136475
The MSE loss is : 0.3048705458641052
The MSE loss is : 0.30487051606178284
The MSE loss is : 0.3048705458641052
The MSE loss is : 0.3048705458641052
The MSE loss is : 0.3048705458641052
The MSE loss is : 0.30487051606178284
The MSE loss is : 0.30487051606178284
The MSE loss is : 0.3048705458641052
The MSE loss is : 0.3048705458641052
The MSE loss is : 0.30487051606178284
The MSE loss is : 0.3048705458641052
The MSE loss is : 0.3048705458641052
The MSE loss is : 0.3048705458641052
The MSE loss is : 0.3048705458641052
Saving... file:record_err_256 method:pair
The MSE loss is : 0.33718395233154297
The MSE loss is : 0.2798923850059509
The MSE loss is : 0.2789155840873718
The MSE loss is : 0.2787536680698395
The MSE loss is : 0.27875101566314

The MSE loss is : 0.14135268330574036
The MSE loss is : 0.14112523198127747
Saving... file:record_err_256 method:pair-Seq

Experiment N=256 SEED=369
The MSE loss is : 0.33567240834236145
The MSE loss is : 0.30574673414230347
The MSE loss is : 0.30459409952163696
The MSE loss is : 0.30430498719215393
The MSE loss is : 0.30381685495376587
The MSE loss is : 0.30363529920578003
The MSE loss is : 0.30363520979881287
The MSE loss is : 0.30363520979881287
The MSE loss is : 0.3036351799964905
The MSE loss is : 0.30363523960113525
The MSE loss is : 0.30363520979881287
The MSE loss is : 0.30363523960113525
The MSE loss is : 0.30363520979881287
The MSE loss is : 0.30363523960113525
The MSE loss is : 0.30363523960113525
The MSE loss is : 0.30363523960113525
The MSE loss is : 0.30363523960113525
The MSE loss is : 0.3036351799964905
The MSE loss is : 0.3036351799964905
The MSE loss is : 0.30363523960113525
Saving... file:record_err_256 method:pair
The MSE loss is : 0.33567240834236145
The MSE loss i

The MSE loss is : 0.14337050914764404
The MSE loss is : 0.1430325210094452
The MSE loss is : 0.1429394781589508
The MSE loss is : 0.1429389864206314
The MSE loss is : 0.14294520020484924
Saving... file:record_err_256 method:pair-Seq

Experiment N=256 SEED=654
The MSE loss is : 0.3369494080543518
The MSE loss is : 0.30652084946632385
The MSE loss is : 0.3048412799835205
The MSE loss is : 0.3045271933078766
The MSE loss is : 0.30430877208709717
The MSE loss is : 0.3043087422847748
The MSE loss is : 0.3043087422847748
The MSE loss is : 0.3043087124824524
The MSE loss is : 0.3043087422847748
The MSE loss is : 0.3043087124824524
The MSE loss is : 0.30430880188941956
The MSE loss is : 0.30430877208709717
The MSE loss is : 0.3043087422847748
The MSE loss is : 0.3043087422847748
The MSE loss is : 0.3043087422847748
The MSE loss is : 0.3043087124824524
The MSE loss is : 0.3043087422847748
The MSE loss is : 0.3043087422847748
The MSE loss is : 0.3043087422847748
The MSE loss is : 0.3043087720870

The MSE loss is : 0.14169561862945557
The MSE loss is : 0.14170658588409424
The MSE loss is : 0.14143608510494232
The MSE loss is : 0.14167745411396027
The MSE loss is : 0.14127948880195618
The MSE loss is : 0.14162319898605347
The MSE loss is : 0.1413271129131317
The MSE loss is : 0.14107997715473175
Saving... file:record_err_256 method:pair-Seq

Experiment N=256 SEED=741
The MSE loss is : 0.33608463406562805
The MSE loss is : 0.30543357133865356
The MSE loss is : 0.3041008710861206
The MSE loss is : 0.3037857413291931
The MSE loss is : 0.30368655920028687
The MSE loss is : 0.3036805987358093
The MSE loss is : 0.3036805987358093
The MSE loss is : 0.3036805987358093
The MSE loss is : 0.3036805987358093
The MSE loss is : 0.3036805987358093
The MSE loss is : 0.3036805987358093
The MSE loss is : 0.3036806285381317
The MSE loss is : 0.3036805987358093
The MSE loss is : 0.3036805987358093
The MSE loss is : 0.3036805987358093
The MSE loss is : 0.3036806285381317
The MSE loss is : 0.303680598

The MSE loss is : 0.142990842461586
The MSE loss is : 0.14292922616004944
The MSE loss is : 0.14271283149719238
The MSE loss is : 0.1425926834344864
The MSE loss is : 0.14250722527503967
The MSE loss is : 0.14250104129314423
The MSE loss is : 0.1422724425792694
The MSE loss is : 0.142215296626091
The MSE loss is : 0.14239487051963806
The MSE loss is : 0.1420397311449051
The MSE loss is : 0.14202061295509338
Saving... file:record_err_256 method:pair-Seq

Experiment N=256 SEED=963
The MSE loss is : 0.33616235852241516
The MSE loss is : 0.30603542923927307
The MSE loss is : 0.3047929108142853
The MSE loss is : 0.3044098913669586
The MSE loss is : 0.3042568266391754
The MSE loss is : 0.30419740080833435
The MSE loss is : 0.30419743061065674
The MSE loss is : 0.30419740080833435
The MSE loss is : 0.3041974604129791
The MSE loss is : 0.3041974604129791
The MSE loss is : 0.30419743061065674
The MSE loss is : 0.30419743061065674
The MSE loss is : 0.30419740080833435
The MSE loss is : 0.3041974

...

The MSE loss is : 0.24348050355911255
The MSE loss is : 0.24151992797851562
The MSE loss is : 0.240486279129982
The MSE loss is : 0.2398311346769333
The MSE loss is : 0.2393900752067566
The MSE loss is : 0.23911970853805542
The MSE loss is : 0.23894217610359192
The MSE loss is : 0.23876596987247467
The MSE loss is : 0.2386452555656433
The MSE loss is : 0.23853203654289246
The MSE loss is : 0.23846158385276794
The MSE loss is : 0.2383832335472107
The MSE loss is : 0.2382993847131729
The MSE loss is : 0.23824942111968994
The MSE loss is : 0.23819899559020996
The MSE loss is : 0.23815178871154785
The MSE loss is : 0.23812557756900787
The MSE loss is : 0.23810677230358124
Saving... file:record_err_1024 method:pair-Add
The MSE loss is : 0.3342490494251251
The MSE loss is : 0.24181880056858063
The MSE loss is : 0.2401392161846161
The MSE loss is : 0.239535853266716
The MSE loss is : 0.23919445276260376
The MSE loss is : 0.23897197842597961
The MSE loss is : 0.23878422379493713
The MSE loss i

Saving... file:record_err_4096 method:block-sqrt-half
The MSE loss is : 0.33348625898361206
The MSE loss is : 0.31396129727363586
The MSE loss is : 0.3138315975666046
The MSE loss is : 0.3138234317302704
The MSE loss is : 0.3138222396373749
The MSE loss is : 0.3138219118118286
The MSE loss is : 0.31382179260253906
The MSE loss is : 0.3138217329978943
The MSE loss is : 0.3138216733932495
The MSE loss is : 0.3138216733932495
The MSE loss is : 0.3138216733932495
The MSE loss is : 0.3138216733932495
The MSE loss is : 0.3138216733932495
The MSE loss is : 0.3138216733932495
The MSE loss is : 0.3138216733932495
The MSE loss is : 0.3138216733932495
The MSE loss is : 0.3138216733932495
The MSE loss is : 0.3138216733932495
The MSE loss is : 0.3138216733932495
The MSE loss is : 0.3138216733932495
Saving... file:record_err_4096 method:block-sqrt
Saving... file:record_err_4096 method:lowR-same-param
Saving... file:record_err_4096 method:lowR-samex2
Saving... file:record_err_4096 method:lowR-half

E

The MSE loss is : 0.31386634707450867
The MSE loss is : 0.31386634707450867
The MSE loss is : 0.31386634707450867
The MSE loss is : 0.31386634707450867
The MSE loss is : 0.31386634707450867
The MSE loss is : 0.31386634707450867
The MSE loss is : 0.31386637687683105
The MSE loss is : 0.31386634707450867
Saving... file:record_err_4096 method:block-sqrt
Saving... file:record_err_4096 method:lowR-same-param
Saving... file:record_err_4096 method:lowR-samex2
Saving... file:record_err_4096 method:lowR-half

Experiment N=4096 SEED=741
The MSE loss is : 0.3335869312286377
The MSE loss is : 0.3304900527000427
The MSE loss is : 0.33026060461997986
The MSE loss is : 0.33017390966415405
The MSE loss is : 0.33013594150543213
The MSE loss is : 0.3301108479499817
The MSE loss is : 0.3300960659980774
The MSE loss is : 0.33008503913879395
The MSE loss is : 0.33007460832595825
The MSE loss is : 0.3300674259662628
The MSE loss is : 0.33006152510643005
The MSE loss is : 0.3300575315952301
The MSE loss is :

The MSE loss is : 0.3336431384086609
The MSE loss is : 0.33054473996162415
The MSE loss is : 0.3303200900554657
The MSE loss is : 0.33023762702941895
The MSE loss is : 0.33019205927848816
The MSE loss is : 0.3301633894443512
The MSE loss is : 0.33014827966690063
The MSE loss is : 0.33013981580734253
The MSE loss is : 0.3301314413547516
The MSE loss is : 0.3301262855529785
The MSE loss is : 0.3301231861114502
The MSE loss is : 0.33012038469314575
The MSE loss is : 0.33011820912361145
The MSE loss is : 0.33011579513549805
The MSE loss is : 0.33011338114738464
The MSE loss is : 0.33011162281036377
The MSE loss is : 0.3301088213920593
The MSE loss is : 0.33010613918304443
The MSE loss is : 0.3301032483577728
The MSE loss is : 0.3301007151603699
Saving... file:record_err_4096 method:pair
The MSE loss is : 0.3336431384086609
The MSE loss is : 0.3178573250770569
The MSE loss is : 0.31760114431381226
The MSE loss is : 0.3175390958786011
The MSE loss is : 0.3175106644630432
The MSE loss is : 0.

The MSE loss is : 0.33008068799972534
The MSE loss is : 0.33007800579071045
Saving... file:record_err_4096 method:pair
The MSE loss is : 0.33361801505088806
The MSE loss is : 0.31783896684646606
The MSE loss is : 0.3175688683986664
The MSE loss is : 0.3174973130226135
The MSE loss is : 0.31747132539749146
The MSE loss is : 0.3174574375152588
The MSE loss is : 0.3174476623535156
The MSE loss is : 0.31744107604026794
The MSE loss is : 0.3174390196800232
The MSE loss is : 0.31743794679641724
The MSE loss is : 0.31743741035461426
The MSE loss is : 0.31743723154067993
The MSE loss is : 0.31743714213371277
The MSE loss is : 0.3174371123313904
The MSE loss is : 0.3174370229244232
The MSE loss is : 0.3174370229244232
The MSE loss is : 0.31743696331977844
The MSE loss is : 0.31743696331977844
The MSE loss is : 0.31743696331977844
The MSE loss is : 0.31743693351745605
Saving... file:record_err_4096 method:block-sqrt-half
The MSE loss is : 0.33361801505088806
The MSE loss is : 0.314075231552124
T

In [21]:
torch.cuda.empty_cache()

In [22]:
int(np.ceil(np.log2(64)))

6

In [23]:
np.log2(256)

8.0

In [24]:
asfsdfasdf

NameError: name 'asfsdfasdf' is not defined

In [ ]:
sum(p.numel() for p in model.parameters())